In [15]:
import numpy as np
import astropy.units as u
from astropy.modeling.blackbody import blackbody_lambda
from astropy.table import Table
from scipy.interpolate import LinearNDInterpolator
from astropy.constants import G as G_const
# from pylcurve.utils import log_g, separation, get_limbdark_params, wd_to_bb_temp

In [20]:
bands = ('u', 'g', 'r', 'i', 'z')
blackbody_interpolators = dict()
filename = '/home/alex/Documents/4th year/PHY480/pylcurve/pylcurve/data/Twd2Tbb.dat'
# filename = pkg_resources.resource_filename('pylcurve', filename)
t = Table.read(filename, format='ascii.tab')

T = t['Teff']
G = t['logg']

for band in bands:
    temps = t['T_BB_' + band]

    coords_in = list(zip(T, G))
    Tbb_coords_out = list(temps)
    blackbody_interpolators[band] = LinearNDInterpolator(coords_in, Tbb_coords_out)

#####################################################################

# load Gianninas 2013 tables into memory
bands = ('u', 'g', 'r', 'i', 'z')
Quad_interpolators = dict()
Claret_interpolators = dict()

for band in bands:
    filename = '/home/alex/Documents/4th year/PHY480/pylcurve/pylcurve/data/ld_coeffs_{}.txt'.format(band)
    # get installed locations
    # filename = pkg_resources.resource_filename('pylcurve', filename)
    t = Table.read(filename, format='cds')

    # data to interpolate over
    T = t['Teff']
    G = t['log(g)']
    # quadratic limb darkening params
    b = t['b']
    c = t['c']
    # claret limb darkening params
    c1 = t['p']
    c2 = t['q']
    c3 = t['r']
    c4 = t['s']

    # should create interpolator class here so other modules can import and use
    # without overhead of creating interpolation each time

    coords_in = list(zip(T, G))
    Quad_coords_out = list(zip(b, c))
    Claret_coords_out = list(zip(c1, c2, c3, c4))

    Quad_interpolators[band] = LinearNDInterpolator(coords_in, Quad_coords_out)
    Claret_interpolators[band] = LinearNDInterpolator(coords_in, Claret_coords_out)

def wd_to_bb_temp(band, t, logg):
    t_bb = blackbody_interpolators[band](t, logg)
    return t_bb
    
def log_g(m, r):
    m = m*u.M_sun
    r = r*u.R_sun
    return np.log10((G_const*m/r/r).to_value(u.cm/u.s/u.s))

def separation(m1, m2, p):
    mt = (m1+m2) * u.M_sun
    p = p * u.d
    acubed = G_const * p**2 * mt / 4 / np.pi**2
    a = acubed ** (1/3)
    return a.to_value(u.R_sun)

def get_limbdark_params(t1, g1, t2, g2, band):
    Quad = dict()
    Claret = dict()
    Quad['ldc1_1'], Quad['ldc1_2'] = Quad_interpolators[band](t1, g1)
    Quad['ldc2_1'], Quad['ldc2_2'] = Quad_interpolators[band](t2, g2)

    Claret['ldc1_1'], Claret['ldc1_2'], Claret['ldc1_3'], Claret['ldc1_4'] = Claret_interpolators[band](t1,g1)
    Claret['ldc2_1'], Claret['ldc2_2'], Claret['ldc2_3'], Claret['ldc2_4'] = Claret_interpolators[band](t2,g2)
    return Quad, Claret

In [36]:
m1 = 0.3367673698833858
m2 = 0.31320949704627943
r1 = 0.0207908551652011
r2 = 0.01882774861838369
t1 = 20578.6859184473
t2 = 10333.949378062876 
per = 0.09986526680760295
t0 = 57460.651009632675
band = 'z'

log_g1 = log_g(m1, r1)
log_g2 = log_g(m2, r2)
a = separation(m1, m2, per)
model_t1 = wd_to_bb_temp(band, t1, log_g1)
model_t2 = wd_to_bb_temp(band, t2, log_g2)
model_r1 = r1/a  # scale from solar radii to separation units
model_r2 = r2/a
# print('q = ', m2/m1)
# print('model zeropoint = ', t0)
# print('model period = ',per)
print('model temps are t1 = ',model_t1)
print(' and t2 = ',model_t2)
print('model radii are r1 = ',model_r1,' and r2 = ',model_r2)
print(get_limbdark_params(t1,log_g1,t2,log_g2,band)[1])


model temps are t1 =  38744.088944971474
 and t2 =  20984.052073338906
model radii are r1 =  0.026496730253514917  and r2 =  0.02399486564926371
{'ldc1_1': 0.6892648284444474, 'ldc1_2': -0.8925952052365252, 'ldc1_3': 0.8949908221692118, 'ldc1_4': -0.33515284844052645, 'ldc2_1': 0.40911969689984706, 'ldc2_2': 0.19321843447556045, 'ldc2_3': -0.15937061615417902, 'ldc2_4': -0.00023808079146706018}
